In [1]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# Download the architecture of ResNet50 with ImageNet weights
base_model = ResNet50(include_top=False, weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/root/anaconda2/envs/tensorflow/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
# Taking the output of the last convoution block in ResNet50
x = base_model.output

In [4]:
# Adding a Global Average Pooling layer
x = GlobalAveragePooling2D()(x)

In [5]:
# Adding a fully connected layer having 1024 neurons
x = Dense(1024, activation='relu')(x)

In [6]:
# Adding a fully connected layer having 2 neurons which will
# give probability of image having either dog or cat
predictions = Dense(2, activation='softmax')(x)

In [7]:
# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Training only top layers i.e. the layers which we have added in the end
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Compiling the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])

In [10]:
# Creating objects for image augmentations
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
# Proving the path of training and test dataset
# Setting the image input size as (224, 224)
# We are using class mode as binary because there are only two classes in our data
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
# Training the model for 5 epochs
model.fit_generator(training_set,
                         steps_per_epoch = 800,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 2000)


Epoch 1/5
799/800 [============================>.] - ETA: 4s - loss: 0.2351 - accuracy: 0.9141

In [ ]:
# We will try to train the last stage of ResNet50
for layer in base_model.layers[0:143]:
  layer.trainable = False

for layer in base_model.layers[143:]:
  layer.trainable = True

In [ ]:
# Training the model for 10 epochs
model.fit_generator(training_set,
                         steps_per_epoch = 800,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 2000)

In [ ]:
# Saving the weights in the current directory
model.save_weights("resnet50_weights.h5")

In [ ]:
# Predicting the final result of image
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('cat_or_dog_test.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)\

In [ ]:
# Expanding the 3-d image to 4-d image.
# The dimensions will be Batch, Height, Width, Channel
test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
# Predicting the final class
result = classifier.predict(test_image)

In [ ]:
# Fetching the class labels
labels = training_set.class_indices
labels = list(labels.items())

In [ ]:
# Printing the final label
for label, i in labels:
    if i == result:
        print("The test image has: ", label)
        break